<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9E%D0%9F%D0%94/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Средний вариант с fuzzy-search

In [ ]:
!pip install rapidfuzz
!pip install spacy

In [ ]:
# extract_and_cluster_colab_L12.py

# Установка зависимостей (нужно запустить один раз)
# pip install sentence-transformers scikit-learn rapidfuzz numpy

import json
import os
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from rapidfuzz import process
import re

# Пути к файлам (в Colab файлы лежат в корне по умолчанию)
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters_L12.json"

# Название модели (загрузится автоматически)
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.65
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2
GEOCODE_SIMILARITY_THRESHOLD = 0.1

# Словарь топонимов Екатеринбурга
# Словарь топонимов Екатеринбурга
# Словарь топонимов Екатеринбурга
TOPONYMS = [
    "Верх-Исетский",
    "Железнодорожный",
    "Кировский",
    "Ленинский",
    "Октябрьский",
    "Орджоникидзевский",
    "Чкаловский",
    "Академический",
    "Вторчермет",
    "Втузгородок",
    "Горный Щит",
    "Елизаветинское",
    "ЖБИ",
    "Завокзальный",
    "Изумрудный",
    "Кольцово",
    "Комсомольский",
    "Короленковский",
    "Малый Исток",
    "Метеогорка",
    "Нижнесвердловский",
    "Новая Сортровка",
    "Новобелореченский",
    "Павелшина",
    "Парковый",
    "Пионерский",
    "Психбольница",
    "Рудничный",
    "Семь Ключей",
    "Сибирский тракт",
    "Синие Камни",
    "Сортировка",
    "Старая Сортровка",
    "Старый Октябрь",
    "Татищева",
    "Уктус",
    "УНЦ",
    "Уралмаш",
    "Центр",
    "Черкасская",
    "Шабровский",
    "Шарташ",
    "Широкая Речка",
    "Эльмаш",
    "Юго-Западный",
    "8 Марта",
    "Амундсена",
    "Авиационная",
    "Академика Павлова",
    "Бабушкина",
    "Бахчиванджи",
    "Белинского",
    "Бориса Ельцина",
    "Братиславская",
    "Бродова",
    "Бульвар Академика Семихатова",
    "Бутюлина",
    "Валдайская",
    "Верхняя Пышма",
    "Вилонова",
    "Водная",
    "Волгоградская",
    "Генеральская",
    "Героев России",
    "Глинки",
    "Гоголя",
    "Горького",
    "Декабристов",
    "Донбасская",
    "Дружининская",
    "Ереванская",
    "Жуковского",
    "Заводская",
    "Зои Космодемьянской",
    "Кирова",
    "Колмогорова",
    "Комсомольская",
    "Короленко",
    "Космонавтов",
    "Крауля",
    "Куйбышева",
    "Лермонтова",
    "Луначарского",
    "Малышева",
    "Мамина-Сибиряка",
    "Маршала Жукова",
    "Машинная",
    "Мельникайте",
    "Металлургов",
    "Мира",
    "Октябрьская",
    "Павла Шпагина",
    "Победы",
    "Пролетарская",
    "Проспект Космонавтов",
    "Радищева",
    "Репина",
    "Розы Люксембург",
    "Сакко и Ванцетти",
    "Свердлова",
    "Сибирский тракт",
    "Софьи Ковалевской",
    "Студенческая",
    "Татищева",
    "Техническая",
    "Тургенева",
    "Туполева",
    "Учительская",
    "Фрунзе",
    "Чапаева",
    "Чкалова",
    "Шарташская",
    "Шефская",
    "Шиловская",
    "Щорса",
    "Энгельса",
    "Площадь 1905 года",
    "Площадь Труда",
    "ЦУМ",
    "Геологическая",
    "ВИЗ",
    "Уралмаш",
    "Сакко",
    "Геолка",
    "1905",
    "Уралмашь",
    "Выксунский завод",
    "Монтажников",
    "Расточная",
    "Екатеринбург",
    "Расточная",
    "Монтажников",
    "УрФУ",
    "НВК",
    "Новокольцовский",
    "НВК"
]

def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())

def extract_geocodes(text: str) -> list[str]:
    text = normalize_text(text)
    found = set()
    for word in text.split():
        match, score, _ = process.extractOne(word, TOPONYMS)
        if score >= 75:
            found.add(match)
    return list(found)

def geocode_similarity(g1: list[str], g2: list[str], threshold=GEOCODE_SIMILARITY_THRESHOLD) -> bool:
    if not g1 or not g2:
        return False
    set1 = set(g1)
    set2 = set(g2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return False
    return (intersection / union) >= threshold

def cluster_messages(messages, model):
    # Добавляем geocode к каждому сообщению
    for msg in messages:
        msg["geocode"] = extract_geocodes(msg["text"])

    # Группируем сообщения по схожести geocode
    clusters_by_geo = []
    for msg in messages:
        assigned = False
        for geo_cluster in clusters_by_geo:
            if geocode_similarity(msg["geocode"], geo_cluster[0]["geocode"]):
                geo_cluster.append(msg)
                assigned = True
                break
        if not assigned:
            clusters_by_geo.append([msg])

    clusters = []
    cluster_id_counter = 1

    for geo_group in clusters_by_geo:
        if len(geo_group) < 2:
            continue

        texts = [m["text"] for m in geo_group]
        embeddings = model.encode(texts, convert_to_numpy=True)

        clustering = DBSCAN(eps=EPS_DBSCAN, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        cluster_map = {}
        for i, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            if label not in cluster_map:
                cluster_map[label] = []
            cluster_map[label].append(geo_group[i])

        for cluster_label, cluster_msgs in cluster_map.items():
            if len(cluster_msgs) < 2:
                continue

            primary_geo = cluster_msgs[0]["geocode"]
            core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
            cluster = {
                "cluster_id": f"cl_{cluster_id_counter:03d}",
                "geocode": primary_geo,
                "message_count": len(cluster_msgs),
                "first_seen": min(m["date"] for m in cluster_msgs),
                "last_seen": max(m["date"] for m in cluster_msgs),
                "core_embedding": core_emb.tolist(),
                "examples": cluster_msgs
            }
            clusters.append(cluster)
            cluster_id_counter += 1

    return clusters

def main():
    print("Загрузка модели L12...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')  # Colab может использовать GPU, если доступно

    print("Загрузка сообщений...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)

    print(f"Обнаружено {len(messages)} сообщений.")

    print("Извлечение геокодов и кластеризация...")
    clusters = cluster_messages(messages, model)

    print(f"Создано {len(clusters)} кластеров.")

    print("Сохранение кластеров...")
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": clusters}, f, ensure_ascii=False, indent=2)

from natasha import Doc, MorphVocab, NewsEmbedding, NewsNERTagger, NewsMorphTagger
from razdel import sentenize, tokenize

def extract_geo_with_natasha(text: str) -> list[str]:
    doc = Doc(text)
    embedding = NewsEmbedding()
    morph_tagger = NewsMorphTagger(embedding)
    ner_tagger = NewsNERTagger(embedding)

    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)

    geos = []
    for span in doc.spans:
        if span.type == 'LOC':  # или 'GPE' — Geopolitical Entity
            geos.append(span.text)
    return geos

def extract_geocodes_enhanced(text: str) -> list[str]:
    found = set()

    # 1. Fuzzy-поиск по словарю
    text_norm = normalize_text(text)
    for word in text_norm.split():
        match, score, _ = process.extractOne(word, TOPONYMS)
        if score >= 85:
            found.add(match)

    # 2. Извлечение через NER (например, natasha)
    ner_geos = extract_geo_with_natasha(text)
    for geo in ner_geos:
        # Сопоставляем с каноническим словарём
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 80:
            found.add(match)

    # 3. Регулярки для паттернов
    patterns = [
        r'на\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "на Ленина"
        r'улица\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "улица Ленина"
        r'ул\.\s+([а-яё]+(?:\s+[а-яё]+)?)',  # "ул. Ленина"
    ]
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for m in matches:
            match, score, _ = process.extractOne(m, TOPONYMS)
            if score >= 85:
                found.add(match)

    return list(found)

    print(f"✅ Обработка завершена. Результат в {CLUSTERS_FILE}")

if __name__ == "__main__":
  extract_geocodes_enhanced("В Кольцово объяснили, почему чемоданы пассажиров валялись на снегу.Как казалось, одна из телег наехала на снежный накат и выронила несколько чемоданов. Водитель автопоезда заметил падение багажа и вернул груз обратно.Отметим, что на взлетно-посадочную полосу багаж, ко")
    # main()

TypeError: 'NoneType' object is not iterable

# Хороший вариант с spacy (без LLM)


In [ ]:
!pip install sentence-transformers scikit-learn rapidfuzz numpy spacy
!python -m spacy download ru_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 97.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# test_cluster_pair.py

from sentence_transformers import SentenceTransformer
import numpy as np

# Название модели
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.5  # Порог косинусной близости

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def test_pair(text1: str, text2: str, model, threshold: float):
    emb1 = model.encode([text1], convert_to_numpy=True)[0]
    emb2 = model.encode([text2], convert_to_numpy=True)[0]

    sim = cosine_similarity(emb1, emb2)
    print(f"Текст 1: {text1}")
    print(f"Текст 2: {text2}")
    print(f"Схожесть: {sim:.4f}")
    print(f"Порог: {threshold}")
    print(f"Результат: {'✅ Объединены' if sim >= threshold else '❌ Не объединены'}")
    print("-" * 50)

def main():
    print("Загрузка модели...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')  # или 'cuda', если GPU доступен

    # Примеры текстов для тестирования
    pairs = [
        (
            "На перекрестке Монтажников и Расточная три машины не поделили дорогу.",
            "ДТП на Монтажников и Расточная. Дорогу не поделили, затор на час."
        ),
        (
            "Свалка мусора на улице Кирова. Нужно убирать.",
            "На Кирова снова свалка! Уже неделю никто не убирает."
        ),
        (
            "Яма на Ленина, 15. Машина повреждена.",
            "Пробка на Монтажников и Расточная."
        ),
        (
            "В Екатеринбурге вынесли приговор банде бывших полицейских.",
            "Суд в центре Екатеринбурга вынес приговор."
        ),
        (
            "В лицее № 110 отменили занятия в начальных классах из-за проведения внеплановой дезинфекции.",
            "Лицей на день закрыл младшие классы для санобработки помещений."
        ),
        (
            "В детском саду № 222 на неделю закрывают группу из-за ремонта санузла.",
            "В лицее № 110 отменили занятия в начальных классах из-за проведения внеплановой дезинфекции."
        ),
        (
            "Светофор не работает на перекрестке 8 Марта - Куйбышева. Водители просят быть внимательнее.",
            "В детском саду № 222 на неделю закрывают группу из-за ремонта санузла."
        ),
        (
            "В гимназии № 35 открылся новый IT-технопарк для уроков робототехники и программирования.",
            "В нашей гимназии открыйти айти-технопарк! Лютая имба!"
        ),
        (
            "В небо над городом запустят праздничный салют в честь Дня города.",
            "В городе прошел традиционный осенний легкоатлетический кросс."
        )
    ]

    print(f"Тестирование пар текстов с порогом {TEXT_SIMILARITY_THRESHOLD}...\n")

    for text1, text2 in pairs:
        test_pair(text1, text2, model, TEXT_SIMILARITY_THRESHOLD)

if __name__ == "__main__":
    main()

Загрузка модели...
Тестирование пар текстов с порогом 0.5...

Текст 1: На перекрестке Монтажников и Расточная три машины не поделили дорогу.
Текст 2: ДТП на Монтажников и Расточная. Дорогу не поделили, затор на час.
Схожесть: 0.6976
Порог: 0.5
Результат: ✅ Объединены
--------------------------------------------------
Текст 1: Свалка мусора на улице Кирова. Нужно убирать.
Текст 2: На Кирова снова свалка! Уже неделю никто не убирает.
Схожесть: 0.4935
Порог: 0.5
Результат: ❌ Не объединены
--------------------------------------------------
Текст 1: Яма на Ленина, 15. Машина повреждена.
Текст 2: Пробка на Монтажников и Расточная.
Схожесть: 0.4813
Порог: 0.5
Результат: ❌ Не объединены
--------------------------------------------------
Текст 1: В Екатеринбурге вынесли приговор банде бывших полицейских.
Текст 2: Суд в центре Екатеринбурга вынес приговор.
Схожесть: 0.6820
Порог: 0.5
Результат: ✅ Объединены
--------------------------------------------------
Текст 1: В лицее № 110 отменили заняти

# Отличный вариант с GigaChat и Spacy

In [ ]:
!pip install gigachat

In [19]:
# Словарь топонимов Екатеринбурга
TOPONYMS = [
    "Верх-Исетский",
    "Железнодорожный",
    "Кировский",
    "Ленинский",
    "Октябрьский",
    "Орджоникидзевский",
    "Чкаловский",
    "Академический",
    "Вторчермет",
    "Втузгородок",
    "Горный Щит",
    "Елизаветинское",
    "ЖБИ",
    "Завокзальный",
    "Изумрудный",
    "Кольцово",
    "Комсомольский",
    "Короленковский",
    "Малый Исток",
    "Метеогорка",
    "Нижнесвердловский",
    "Новая Сортровка",
    "Новобелореченский",
    "Павелшина",
    "Парковый",
    "Пионерский",
    "Психбольница",
    "Рудничный",
    "Семь Ключей",
    "Сибирский тракт",
    "Синие Камни",
    "Сортировка",
    "Старая Сортровка",
    "Старый Октябрь",
    "Татищева",
    "Уктус",
    "УНЦ",
    "Уралмаш",
    "Центр",
    "Черкасская",
    "Шабровский",
    "Шарташ",
    "Широкая Речка",
    "Эльмаш",
    "Юго-Западный",
    "8 Марта",
    "Амундсена",
    "Авиационная",
    "Академика Павлова",
    "Бабушкина",
    "Бахчиванджи",
    "Белинского",
    "Бориса Ельцина",
    "Братиславская",
    "Бродова",
    "Бульвар Академика Семихатова",
    "Бутюлина",
    "Валдайская",
    "Верхняя Пышма",
    "Вилонова",
    "Водная",
    "Волгоградская",
    "Генеральская",
    "Героев России",
    "Глинки",
    "Гоголя",
    "Горького",
    "Декабристов",
    "Донбасская",
    "Дружининская",
    "Ереванская",
    "Жуковского",
    "Заводская",
    "Зои Космодемьянской",
    "Кирова",
    "Колмогорова",
    "Комсомольская",
    "Короленко",
    "Космонавтов",
    "Крауля",
    "Куйбышева",
    "Лермонтова",
    "Луначарского",
    "Малышева",
    "Мамина-Сибиряка",
    "Маршала Жукова",
    "Машинная",
    "Мельникайте",
    "Металлургов",
    "Мира",
    "Октябрьская",
    "Павла Шпагина",
    "Победы",
    "Пролетарская",
    "Проспект Космонавтов",
    "Радищева",
    "Репина",
    "Розы Люксембург",
    "Сакко и Ванцетти",
    "Свердлова",
    "Сибирский тракт",
    "Софьи Ковалевской",
    "Студенческая",
    "Татищева",
    "Техническая",
    "Тургенева",
    "Туполева",
    "Учительская",
    "Фрунзе",
    "Чапаева",
    "Чкалова",
    "Шарташская",
    "Шефская",
    "Шиловская",
    "Щорса",
    "Энгельса",
    "Площадь 1905 года",
    "Площадь Труда",
    "ЦУМ",
    "Геологическая",
    "ВИЗ",
    "Уралмаш",
    "Сакко",
    "Геолка",
    "1905",
    "Уралмашь",
    "Выксунский завод",
    "Ленина",
    "Екатеринбург",
    "Расточная",
    "Монтажников",
    "УрФУ",
    "НВК",
    "Новокольцовский",
    "Кирова",
    "Лицей",
    "Детский",
    "Детсад",
    "Школа",
    "Больница",
    "Университет"
]

Первая итерация - простая кластеризация

In [ ]:
# extract_and_cluster_colab_spacy.py

# Установка зависимостей (нужно запустить один раз)
# pip install sentence-transformers scikit-learn rapidfuzz numpy spacy
# python -m spacy download ru_core_news_sm

import json
import os
import numpy as np
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from rapidfuzz import process
import re

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole
from google.colab import userdata

# Пути к файлам
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters_spacy.json"

# Название модели
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Параметры
TEXT_SIMILARITY_THRESHOLD = 0.45
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2
GEOCODE_SIMILARITY_THRESHOLD = 0.5

AUTH = userdata.get('SBER_AUTH')  # Получить в кабинете СберCloud
# Инициализируем клиент (будем использовать в контексте)
GIGA_MODEL = "GigaChat-2"  # или "GigaChat-Pro"



def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())


def extract_geo_with_gigachat(text: str, auth=AUTH, model=GIGA_MODEL) -> dict:
    """
    Извлекает структурированную геоинформацию из текста с помощью GigaChat SDK.
    Возвращает словарь в формате JSON.
    """
    SYSTEM_PROMPT = (
        "Ты — эксперт по анализу жалоб жителей. Твоя задача — извлечь из текста "
        "географическую информацию (или похожую на такую) и вернуть её в виде строго структурированного JSON. "
        "Не добавляй пояснений, не изменяй формат, не придумывай данные."
    )

    USER_PROMPT = f"""
    Текст сообщения: "{text[0:150]}"
    Извлеки следующие поля в формате JSON:
    {{
      "geocode": ["улица Ленина, 45", "Советская ул., д.10", "школа №3", "Центральный район", "УрФУ"]
    }}

    geocode - названия любых улиц, районов, зданий, мест или иных меток для идентификации положения.
    """

    messages = [
        Messages(role=MessagesRole.SYSTEM, content=SYSTEM_PROMPT),
        Messages(role=MessagesRole.USER, content=USER_PROMPT),
    ]

    payload = Chat(
        messages=messages,
        temperature=0.1,
        max_tokens=300,
    )

    try:
        with GigaChat(credentials=auth, model=model, verify_ssl_certs=False) as giga:
            response = giga.chat(payload)
            raw_text = response.choices[0].message.content.strip()

            # Очистка от возможных markdown-блоков
            if raw_text.startswith("```"):
                raw_text = raw_text.split("```")[1] if "```" in raw_text else raw_text
            if raw_text.startswith("json"):
                raw_text = raw_text[4:].strip()

            import json
            return json.loads(raw_text)
    except Exception as e:
        print(f"[GigaChat Geo] Ошибка при обработке текста '{text[:50]}...': {e}")
        return {
            "geocodes": []
        }


def extract_geo_with_spacy(text: str) -> list[str]:
    try:
        import spacy

        # Проверим, не пустой ли текст
        if not text or len(text.strip()) < 5:
            return []

        nlp = spacy.load("ru_core_news_sm")
        doc = nlp(text)

        geos = []
        for ent in doc.ents:
            if ent.label_ in ["LOC", "GPE", "FAC"]:  # Location, Geopolitical Entity, Facility
                geos.append(ent.text)
        return geos
    except Exception as e:
        print(f"Ошибка при извлечении геокода через spacy: {e}")
        return []

def extract_geocodes_enhanced(text: str) -> list[str]:
    found = set()

    # 2. Извлечение через NER (spacy)
    ner_geos = extract_geo_with_spacy(text)
    for geo in ner_geos:
        # Сопоставляем с каноническим словарём
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 50:
          found.add(match)

    # print(found)
    # Если не смогли найти геокоды с spacy, вызываем GigaChat
    if len(found) == 0:
        found = extract_geo_with_gigachat(text)
        return list(found["geocode"])

    return list(found)

def geocode_similarity(g1: list[str], g2: list[str], threshold=GEOCODE_SIMILARITY_THRESHOLD) -> bool:
    if not g1 or not g2:
        return False
    set1 = set(g1)
    set2 = set(g2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    if union == 0:
        return False
    return (intersection / union) >= threshold

def cluster_messages(messages, model):
    # Добавляем geocode к каждому сообщению
    for msg in messages:
        msg["geocode"] = extract_geocodes_enhanced(msg["text"])

    # Группируем сообщения по схожести geocode
    clusters_by_geo = []
    for msg in messages:
        assigned = False
        for geo_cluster in clusters_by_geo:
            if geocode_similarity(msg["geocode"], geo_cluster[0]["geocode"]):
                geo_cluster.append(msg)
                assigned = True
                break
        if not assigned:
            clusters_by_geo.append([msg])

    clusters = []
    cluster_id_counter = 1

    for geo_group in clusters_by_geo:
        if len(geo_group) < 2:
            continue

        texts = [m["text"] for m in geo_group]
        embeddings = model.encode(texts, convert_to_numpy=True)

        clustering = DBSCAN(eps=EPS_DBSCAN, min_samples=MIN_SAMPLES, metric='cosine').fit(embeddings)

        cluster_map = {}
        for i, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            if label not in cluster_map:
                cluster_map[label] = []
            cluster_map[label].append(geo_group[i])

        for cluster_label, cluster_msgs in cluster_map.items():
            if len(cluster_msgs) < 2:
                continue

            primary_geo = cluster_msgs[0]["geocode"]
            core_emb = np.mean(model.encode([m["text"] for m in cluster_msgs], convert_to_numpy=True), axis=0)
            cluster = {
                "cluster_id": f"cl_{cluster_id_counter:03d}",
                "geocode": primary_geo,
                "message_count": len(cluster_msgs),
                "first_seen": min(m["date"] for m in cluster_msgs),
                "last_seen": max(m["date"] for m in cluster_msgs),
                "core_embedding": core_emb.tolist(),
                "examples": cluster_msgs
            }
            clusters.append(cluster)
            cluster_id_counter += 1

    return clusters

def main():
    print("Загрузка модели L12...")
    model = SentenceTransformer(MODEL_NAME, device='cpu')

    print("Загрузка сообщений...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)

    print(f"Обнаружено {len(messages)} сообщений.")

    print("Извлечение геокодов и кластеризация...")
    clusters = cluster_messages(messages, model)

    print(f"Создано {len(clusters)} кластеров.")

    print("Сохранение кластеров...")
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": clusters}, f, ensure_ascii=False, indent=2)

    print(f"✅ Обработка завершена. Результат в {CLUSTERS_FILE}")

if __name__ == "__main__":
    # print(extract_geo_with_gigachat("Власти объявили о расширении Режевского тракта до 4 полос в следующем году."))
    main()

Загрузка модели L12...
Загрузка сообщений...
Обнаружено 10 сообщений.
Извлечение геокодов и кластеризация...
set()
{'Расточная'}
{'Расточная'}
set()
set()
{'Ленина'}
{'Ленина'}
{'Машинная'}
set()
{'Кирова'}
Создано 2 кластеров.
Сохранение кластеров...
✅ Обработка завершена. Результат в clusters_spacy.json


Добавляем id сообщениям

In [61]:
import json
import hashlib
import os

# Путь к файлу
INPUT_FILE = "messages.json"

# Проверка существования
if not os.path.exists(INPUT_FILE):
    print(f"❌ Файл {INPUT_FILE} не найден. Загрузите его в Colab.")
    exit()

# Чтение
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    messages = json.load(f)

print(f"📥 Загружено {len(messages)} сообщений.")

# Добавление ID
updated = 0
for msg in messages:
    if "id" not in msg or not msg["id"]:
        # Используем URL как основу (лучший идентификатор для Telegram)
        url = msg.get("url", "").strip()
        if url:
            # Убираем пробелы и параметры после '?' (на всякий случай)
            clean_url = url.split('?')[0].strip()
            # Генерируем короткий хэш (16 символов, как в вашем примере)
            msg_id = hashlib.sha256(clean_url.encode('utf-8')).hexdigest()[:16]
        else:
            # Fallback: хэш от текста + даты
            fallback = msg.get("text", "") + msg.get("date", "")
            msg_id = hashlib.md5(fallback.encode('utf-8')).hexdigest()[:16]

        msg["id"] = msg_id
        updated += 1

print(f"🆕 Добавлено ID для {updated} сообщений.")

# Сохранение
with open(INPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(messages, f, ensure_ascii=False, indent=2)

print(f"✅ Файл {INPUT_FILE} обновлён.")

📥 Загружено 1 сообщений.
🆕 Добавлено ID для 1 сообщений.
✅ Файл messages.json обновлён.


# Вторая итерация - инкрементная кластеризация

Важно: подгрузите перед началом топонимы из блоков выше

А ещё у всех сообщений должен быть id. Чекните код выше, там можно сгенерировать их всем сообщениям в вашем датасете

In [62]:
# УСТАНОВКА ЗАВИСИМОСТЕЙ (запустите один раз)
!pip install -q sentence-transformers scikit-learn rapidfuzz spacy geopy gigachat

# ЗАГРУЗКА МОДЕЛИ SPACY (один раз)
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "spacy", "download", "ru_core_news_sm"])

# ИМПОРТЫ
import json
import os
import re
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from rapidfuzz import process
import spacy

# GigaChat
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole
from google.colab import userdata

# Geocoding
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# ======================
# КОНФИГУРАЦИЯ
# ======================
MESSAGES_FILE = "messages.json"
CLUSTERS_FILE = "clusters.json"
CACHE_DIR = "cache"
os.makedirs(CACHE_DIR, exist_ok=True)

GEO_CACHE_FILE = os.path.join(CACHE_DIR, "geocodes_cache.json")
PROCESSED_IDS_FILE = os.path.join(CACHE_DIR, "processed_ids.json")
NOISE_BUFFER_FILE = os.path.join(CACHE_DIR, "noise_buffer.json")

# Модель эмбеддингов
MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
TEXT_SIMILARITY_THRESHOLD = 0.45
EPS_DBSCAN = 1 - TEXT_SIMILARITY_THRESHOLD
MIN_SAMPLES = 2

# TTL для буфера выбросов (часы)
NOISE_TTL_HOURS = 24
# Максимальный размер буфера выбросов
MAX_NOISE_BUFFER_SIZE = 200

# Геокодер (OpenStreetMap)
geolocator = Nominatim(user_agent="ekb_municipal_ai_hackathon")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Топонимы выше
# .......

# GigaChat
AUTH = userdata.get('SBER_AUTH')
GIGA_MODEL = "GigaChat"

# ======================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ======================

def normalize_text(text: str) -> str:
    return re.sub(r'[^\w\s\dа-яё]', ' ', text.lower())

def extract_geo_with_spacy(text: str) -> list[str]:
    try:
        nlp = spacy.load("ru_core_news_sm")
        doc = nlp(text)
        return [ent.text for ent in doc.ents if ent.label_ in ("LOC", "GPE", "FAC")]
    except Exception as e:
        print(f"[spaCy] Ошибка: {e}")
        return []

def extract_geo_with_gigachat(text: str) -> list[str]:
    SYSTEM_PROMPT = (
        "Ты — эксперт по анализу жалоб жителей Екатеринбурга. "
        "Извлеки из текста географические объекты (улицы, районы, здания, площади) "
        "и верни ТОЛЬКО JSON-массив в поле 'geocode'."
    )
    USER_PROMPT = f"""
      Текст сообщения: "{text[:150]}"
      Примеры для твоих инструкций.
      Вход: "Вчера в бассейне 'Юность' произошла"
      Выход: {{"geocode": ["Юность", "бассейн"]}}
      Вход: "На перекрестке Монтажников - Расточная авария",
      Выход: {{"geocode": ["Монтажников", "Расточная"]}}
      Вход: "Ремонт на улице Монтажников затянулся."
      Выход: {{"geocode": ["Монтажников"]}}
      Вход: "Дети из СОШ №3 жалуются на отопление."
      Выход: {{"geocode": ["школа №3"]}}
      Вход: "Вчера в 'Юности' обедали"
      Выход: {{"geocode": ["Юность", "обед"]}}
      Все формы (даже в кавычках) возвращай в именительном падеже
    """

    messages = [
        Messages(role=MessagesRole.SYSTEM, content=SYSTEM_PROMPT),
        Messages(role=MessagesRole.USER, content=USER_PROMPT),
    ]

    payload = Chat(messages=messages, temperature=0.1, max_tokens=200)

    try:
        with GigaChat(credentials=AUTH, model=GIGA_MODEL, verify_ssl_certs=False) as giga:
            resp = giga.chat(payload)
            content = resp.choices[0].message.content.strip()
            if content.startswith("```"):
                content = content.split("```")[1] if "```" in content else content
            if content.startswith("json"):
                content = content[4:].strip()
            data = json.loads(content)
            return data.get("geocode", [])
    except Exception as e:
        print(f"[GigaChat] Ошибка: {e}")
        return []

def geocode_toponyms(toponyms: list[str], city="Екатеринбург") -> list[dict]:
    results = []
    for topo in toponyms:
        query = f"{topo}, {city}"
        try:
            loc = geocode(query, country_codes="RU", timeout=10)
            if loc:
                results.append({
                    "name": topo,
                    "lat": round(loc.latitude, 6),
                    "lon": round(loc.longitude, 6)
                })
            else:
                results.append({"name": topo, "lat": None, "lon": None})
        except Exception as e:
            print(f"[Nominatim] {e}")
            results.append({"name": topo, "lat": None, "lon": None})
    return results

def extract_geocodes_cached(msg_id: str, text: str, geocache: dict) -> dict:
    if msg_id in geocache:
        return geocache[msg_id]

    # 1. spaCy + словарь
    found = set()
    ner_geos = extract_geo_with_spacy(text)
    for geo in ner_geos:
        match, score, _ = process.extractOne(geo, TOPONYMS)
        if score >= 50:
            found.add(match)

    source = "spacy"
    if not found:
        found = set(extract_geo_with_gigachat(text))
        source = "gigachat"

    found = list(found)
    coords = geocode_toponyms(found) if found else []

    result = {
        "geocode": found,
        "coords": coords,
        "source": source,
        "timestamp": datetime.now().isoformat()
    }
    geocache[msg_id] = result
    return result

def load_cache():
    geocache = {}
    processed_ids = set()
    if os.path.exists(GEO_CACHE_FILE):
        with open(GEO_CACHE_FILE, "r", encoding="utf-8") as f:
            geocache = json.load(f)
    if os.path.exists(PROCESSED_IDS_FILE):
        with open(PROCESSED_IDS_FILE, "r", encoding="utf-8") as f:
            processed_ids = set(json.load(f))
    return geocache, processed_ids

def save_cache(geocache, processed_ids):
    with open(GEO_CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(geocache, f, ensure_ascii=False, indent=2)
    with open(PROCESSED_IDS_FILE, "w", encoding="utf-8") as f:
        json.dump(list(processed_ids), f, ensure_ascii=False, indent=2)

def load_noise_buffer():
    if os.path.exists(NOISE_BUFFER_FILE):
        with open(NOISE_BUFFER_FILE, "r", encoding="utf-8") as f:
            buffer = json.load(f)
        now = datetime.now()
        return [
            msg for msg in buffer
            if (now - datetime.fromisoformat(msg["buffered_at"])) < timedelta(hours=NOISE_TTL_HOURS)
        ]
    return []

def save_noise_buffer(buffer):
    with open(NOISE_BUFFER_FILE, "w", encoding="utf-8") as f:
        json.dump(buffer, f, ensure_ascii=False, indent=2)

from sklearn.metrics.pairwise import cosine_similarity

def load_existing_clusters():
    if not os.path.exists(CLUSTERS_FILE):
        return []
    try:
        with open(CLUSTERS_FILE, "r", encoding="utf-8") as f:
            data = json.load(f)
        clusters = data.get("clusters", [])
        # Конвертируем эмбеддинги в numpy для сравнения
        for cl in clusters:
            if isinstance(cl["core_embedding"], list):
                cl["core_embedding"] = np.array(cl["core_embedding"], dtype=np.float32)
        return clusters
    except Exception as e:
        print(f"[WARN] Не удалось загрузить кластеры: {e}")
        return []

def save_clusters(clusters):
    # Преобразуем эмбеддинги обратно в списки для JSON
    serializable = []
    for cl in clusters:
        cl_copy = cl.copy()
        if isinstance(cl_copy["core_embedding"], np.ndarray):
            cl_copy["core_embedding"] = cl_copy["core_embedding"].tolist()
        serializable.append(cl_copy)
    with open(CLUSTERS_FILE, "w", encoding="utf-8") as f:
        json.dump({"clusters": serializable}, f, ensure_ascii=False, indent=2)

def update_cluster_with_message(cluster, new_msg, model, max_examples=5):
    # Собираем все тексты: старые примеры + новое сообщение
    all_texts = [ex["text"] for ex in cluster["examples"]] + [new_msg["text"]]
    new_emb = np.mean(model.encode(all_texts, convert_to_numpy=True), axis=0)

    # Обновляем примеры — оставляем не более max_examples, приоритет свежим
    updated_examples = (cluster["examples"] + [new_msg])[-max_examples:]

    cluster.update({
        "core_embedding": new_emb,
        "message_count": cluster["message_count"] + 1,
        "last_seen": max(cluster["last_seen"], new_msg["date"]),
        "examples": updated_examples
    })

def try_assign_to_existing_clusters(new_messages, existing_clusters, model, geo_threshold=0.5, sim_threshold=0.45):
    unassigned = []
    for msg in new_messages:
        assigned = False
        msg_emb = model.encode([msg["text"]], convert_to_numpy=True)[0]

        for cl in existing_clusters:
            # Географическое совпадение (твоя функция, без изменений)
            if not geocode_similarity(msg["geocode"], cl["geocode"], threshold=geo_threshold):
                continue

            # Семантическое сходство
            sim = cosine_similarity([msg_emb], [cl["core_embedding"]])[0][0]
            if sim >= sim_threshold:
                update_cluster_with_message(cl, msg, model)
                assigned = True
                break

        if not assigned:
            unassigned.append(msg)
    return unassigned

# ======================
# КЛАСТЕРИЗАЦИЯ
# ======================

def geocode_similarity(g1, g2, threshold=0.5):
    if not g1 or not g2:
        return False
    s1, s2 = set(g1), set(g2)
    return len(set(s1 & s2)) / len(set(s1 | s2)) >= threshold

def cluster_with_geo_grouping(messages, model):
    # Геогруппировка
    geo_groups = []
    for msg in messages:
        placed = False
        for group in geo_groups:
            if geocode_similarity(msg["geocode"], group[0]["geocode"]):
                group.append(msg)
                placed = True
                break
        if not placed:
            geo_groups.append([msg])

    clusters, noise = [], []
    for group in geo_groups:
        if len(group) == 1:
            noise.extend(group)
            continue

        # Семантическая кластеризация внутри геогруппы
        texts = [m["text"] for m in group]
        emb = model.encode(texts, convert_to_numpy=True)
        labels = DBSCAN(eps=EPS_DBSCAN, min_samples=MIN_SAMPLES, metric='cosine').fit(emb).labels_

        clustered_ids = set()
        for label in set(labels):
            if label == -1:
                continue
            msgs = [group[i] for i, l in enumerate(labels) if l == label]
            if len(msgs) >= 2:
                core_emb = np.mean(model.encode([m["text"] for m in msgs], convert_to_numpy=True), axis=0)
                clusters.append({
                    "cluster_id": f"cl_{len(clusters)+1:03d}",
                    "geocode": msgs[0]["geocode"],
                    "coords": msgs[0]["coords"],
                    "message_count": len(msgs),
                    "first_seen": min(m["date"] for m in msgs),
                    "last_seen": max(m["date"] for m in msgs),
                    "core_embedding": core_emb.tolist(),
                    "examples": msgs
                })
                clustered_ids.update(m["id"] for m in msgs)

        # Не вошедшие в кластеры — в шум
        noise.extend([m for m in group if m["id"] not in clustered_ids])

    return clusters, noise

# ======================
# MAIN
# ======================

def main():
    print("🚀 Запуск AI-агента для Главы Екатеринбурга...")
    with open(MESSAGES_FILE, "r", encoding="utf-8") as f:
        messages = json.load(f)
    if not messages:
        return

    # Загрузка модели
    model = SentenceTransformer(MODEL_NAME, device='cpu')

    # Кэш обработанных сообщений
    geocache, processed_ids = load_cache()
    new_messages = [m for m in messages if m["id"] not in processed_ids]
    print(f"🆕 Новых сообщений: {len(new_messages)}")

    if not new_messages:
        print("💤 Нет новых данных.")
        return

    # Геокодирование новых сообщений
    for msg in new_messages:
        geo_data = extract_geocodes_cached(msg["id"], msg["text"], geocache)
        msg["geocode"] = geo_data["geocode"]
        msg["coords"] = geo_data["coords"]

    for msg in new_messages:
        processed_ids.add(msg["id"])
    save_cache(geocache, processed_ids)

    # === ИНКРЕМЕНТАЛЬНОЕ ОБНОВЛЕНИЕ КЛАСТЕРОВ ===
    # 1. Загружаем существующие кластеры
    existing_clusters = load_existing_clusters()

    # 2. Пробуем присоединить новые сообщения к существующим кластерам
    remaining_new = try_assign_to_existing_clusters(
        new_messages,
        existing_clusters,
        model,
        geo_threshold=0.3,
        sim_threshold=TEXT_SIMILARITY_THRESHOLD  # используем твой порог
    )

    # 3. Загружаем буфер выбросов и объединяем с неприсоединёнными новыми
    noise_buffer = load_noise_buffer()
    all_for_clustering = remaining_new + noise_buffer

    # 4. Кластеризуем только то, что не попало в старые кластеры
    new_clusters, remaining_noise = cluster_with_geo_grouping(all_for_clustering, model)

    # 5. Объединяем обновлённые старые + новые кластеры
    all_clusters = existing_clusters + new_clusters

    # 6. Обновляем буфер выбросов
    now = datetime.now().isoformat()
    for msg in remaining_noise:
        msg["buffered_at"] = now
    save_noise_buffer(remaining_noise)

    # 7. Сохраняем ВСЕ кластеры (старые + новые)
    save_clusters(all_clusters)

    print(f"\n✅ Готово!")
    print(f"  • Обновлено/создано кластеров: {len(all_clusters)}")
    print(f"  • В буфере: {len(remaining_noise)} (TTL={NOISE_TTL_HOURS}ч)")
    print(f"  • Результат: {CLUSTERS_FILE}")
if __name__ == "__main__":
    main()

🚀 Запуск AI-агента для Главы Екатеринбурга...
🆕 Новых сообщений: 1

✅ Готово!
  • Обновлено/создано кластеров: 13
  • В буфере: 45 (TTL=24ч)
  • Результат: clusters.json


# Тестируем GigaChat на географию

In [ ]:
# УСТАНОВКА ЗАВИСИМОСТЕЙ (запустите один раз)
!pip install gigachat

In [46]:
# GigaChat
import json
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole
from google.colab import userdata


# Топонимы выше
# .......

# GigaChat
AUTH = userdata.get('SBER_AUTH')
GIGA_MODEL = "GigaChat-2"

def extract_geo_with_gigachat(text: str) -> list[str]:
    SYSTEM_PROMPT = (
        "Ты — эксперт по анализу жалоб жителей Екатеринбурга. "
        "Извлеки из текста географические объекты (улицы, районы, здания, площади) "
        "и верни ТОЛЬКО JSON-массив в поле 'geocode'."
    )
    USER_PROMPT = f"""
      Текст сообщения: "{text[:150]}"
      Примеры для твоих инструкций.
      Вход: "Вчера в бассейне 'Юность' произошла"
      Выход: {{"geocode": ["Юность", "бассейн"]}}
      Вход: "На перекрестке Монтажников - Расточная авария",
      Выход: {{"geocode": ["Монтажников", "Расточная"]}}
      Вход: "Ремонт на улице Монтажников затянулся."
      Выход: {{"geocode": ["Монтажников"]}}
      Вход: "Дети из СОШ №3 жалуются на отопление."
      Выход: {{"geocode": ["школа №3"]}}
      Вход: "Вчера в 'Юности' обедали"
      Выход: {{"geocode": ["Юность", "обед"]}}
      Все формы (даже в кавычках) возвращай в именительном падеже. Игнорируй названия стран, городов
    """
    print(USER_PROMPT)

    messages = [
        Messages(role=MessagesRole.SYSTEM, content=SYSTEM_PROMPT),
        Messages(role=MessagesRole.USER, content=USER_PROMPT),
    ]

    payload = Chat(messages=messages, temperature=0.1, max_tokens=200)

    try:
        with GigaChat(credentials=AUTH, model=GIGA_MODEL, verify_ssl_certs=False) as giga:
            resp = giga.chat(payload)
            content = resp.choices[0].message.content.strip()
            if content.startswith("```"):
                content = content.split("```")[1] if "```" in content else content
            if content.startswith("json"):
                content = content[4:].strip()
            data = json.loads(content)
            return data.get("geocode", [])
    except Exception as e:
        print(f"[GigaChat] Ошибка: {e}")
        return []

print(extract_geo_with_gigachat("На линии Екатеринбург - Первоуральск начал курсировать обновленный состав."))



      Текст сообщения: "На линии Екатеринбург - Первоуральск начал курсировать обновленный состав."
      Примеры для твоих инструкций.
      Вход: "Вчера в бассейне 'Юность' произошла"
      Выход: {"geocode": ["Юность", "бассейн"]}
      Вход: "На перекрестке Монтажников - Расточная авария",
      Выход: {"geocode": ["Монтажников", "Расточная"]}
      Вход: "Ремонт на улице Монтажников затянулся."
      Выход: {"geocode": ["Монтажников"]}
      Вход: "Дети из СОШ №3 жалуются на отопление."
      Выход: {"geocode": ["школа №3"]}
      Вход: "Вчера в 'Юности' обедали"
      Выход: {"geocode": ["Юность", "обед"]}
      Все формы (даже в кавычках) возвращай в именительном падеже.
    
['Екатеринбург', 'Первоуральск']


In [69]:
me = {1, 2}
me2 = {1, 3}
print(me | me2)

{1, 2, 3}
